In [15]:
import json
import matplotlib.pyplot as plt
import numpy as np
import cv2
from joblib import Parallel, delayed
import os
from tqdm import tqdm

In [11]:
def load_images(file):
    return np.array(cv2.imreadmulti(file, flags=cv2.IMREAD_GRAYSCALE)[1])

In [12]:
def process_images(images):
    def get_mask(image, back):
        return np.abs(image - back) / np.max(image) > 0.1

    back = np.mean(images, axis=0)
    masks = Parallel(n_jobs=8)(delayed(get_mask)(image,back) for image in images)
    masks = np.array(masks)
    masks = masks.astype(np.uint8) * 255
    return masks

In [13]:
def get_contours(masks, frame):
    def clean_frame(mask):
        image = mask.copy()
        image = cv2.medianBlur(image, 3)
        
        kernel = np.ones((3,3),np.uint8)
        image = cv2.dilate(image, kernel, iterations=9)

        image = cv2.medianBlur(image, 17)

        contours, _ = cv2.findContours(image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        image = np.zeros(image.shape)
        bb = []
        for contour in contours:
            if cv2.contourArea(contour) > 300:
                cv2.drawContours(image, [contour], 0, (255), -1)
                x, y, w, h = cv2.boundingRect(contour)
                bb.append((x, y, w, h))
                cv2.rectangle(image, (x, y), (x+w, y+h), (255), 2)

    
        kernel = np.ones((100,100),np.uint8)
        image = cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)
        image = np.array(image, dtype=np.uint8)
        return image

    def get_boundaries(mask):
        boundaries = np.zeros(mask.shape)
        contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        for contour in contours:
            if cv2.contourArea(contour) > 300:
                cv2.drawContours(boundaries, [contour], 0, (255), 5)
        return np.array(boundaries, dtype=np.uint8)

    def get_circles(mask):
        return cv2.HoughCircles(mask, cv2.HOUGH_GRADIENT, dp=1, minDist=500, param1=50, param2=20, minRadius=270, maxRadius=350)

    def get_bounding_boxes(mask):
        circles = get_circles(mask)
        bb = []
        if circles is not None:
            circles = np.uint16(np.around(circles))
            for circle in circles[0,:]:
                x, y, r = circle
                x = int(x)
                y = int(y)
                r = int(r)
                if x-r < 0 or y-r < 0 or x+r > mask.shape[1] or y+r > mask.shape[0]:
                    continue
                bb.append((x-r, y-r, 2*r, 2*r))
        return bb
    
    mask = masks[frame-1]
    mask = clean_frame(mask)
    mask = get_boundaries(mask)
    bb   = get_bounding_boxes(mask)
    return mask, bb       


In [28]:
def file_to_droplets(file):
    images = load_images(file)
    print(f'Processing {file} with {len(images)} frames')
    masks  = process_images(images)
    print(f'Extracting droplets from {file}')
    for frame in tqdm(range(1, 31)):
        droplets = list()
        _ , bb = get_contours(masks, frame)
        for x, y, w, h in bb:
            droplets.append(masks[frame-1][y:y+h, x:x+w])
        file_name = file.split("/")[-1].split(".")[0]+"-"+str(frame).replace("-", "_")
        file_name = f'./data/droplets/{file_name}'
        cv2.imwritemulti(file_name+'.tif', droplets)
        with open(file_name+'.json', 'w') as f:
            json.dump(bb, f)
    print(f'Finished {file}')

file_list = os.listdir('./data/sequences')

for file in tqdm(file_list):
    file_to_droplets(f'./data/sequences/{file}')


  0%|          | 0/41 [00:00<?, ?it/s]

Processing ./data/sequences/1-1.tif with 100 frames
Extracting droplets from ./data/sequences/1-1.tif


  2%|▏         | 1/41 [00:39<26:24, 39.62s/it]

Finished ./data/sequences/1-1.tif
Processing ./data/sequences/1-2.tif with 100 frames
Extracting droplets from ./data/sequences/1-2.tif


  5%|▍         | 2/41 [01:08<21:32, 33.15s/it]

...

Finished ./data/sequences/9-1.tif
Processing ./data/sequences/9-2.tif with 100 frames
Extracting droplets from ./data/sequences/9-2.tif


 98%|█████████▊| 40/41 [21:17<00:29, 29.16s/it]

Finished ./data/sequences/9-2.tif
Processing ./data/sequences/9-3.tif with 100 frames
Extracting droplets from ./data/sequences/9-3.tif


100%|██████████| 41/41 [21:46<00:00, 31.88s/it]

Finished ./data/sequences/9-3.tif


In [38]:
group = 13
student = 2
frame = 23

file = f'{group}-{student}-{frame}'

images = load_images('./data/droplets/'+file+'.tif')
with open('./data/droplets/'+file+'.json', 'r') as f:
    bb = json.load(f)

for i in range(len(bb)):
    print(bb[i][2:])
    print(images[i].shape)

[634, 634]
(634, 634)
[646, 646]
(646, 646)


C:\Users\lambe\AppData\Local\Temp\ipykernel_23380\810626805.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(cv2.imreadmulti(file, flags=cv2.IMREAD_GRAYSCALE)[1])
